# Глава 28. Подробнее о программировании классов

## Инструкция `class`

Инструкция `class` не является объявлением, подобно `def` она создает объект и является неявной инструкцией присваивания - когда он выполняется, создается объект класса, ссылка на который сохраняется в имени, использованном в заголовке инструкции.

Также `class` является настоящим выполняемым программным кодом - класс не существует, пока поток выполнения не достигнет инструкции `class`, которая определяет его (обычно при импортировании модуля, в котором она находится, но не ранее).

### Общая форма

```
class <name>(superclass,...): # Присваивание имени
    data = value              # Совместно исп-ые данные класса
    def method(self,...):     # Методы
        self.member = value   # Данные экземпляров
```

### Пример

* Подобно функциям, инструкции `class` являются локальными областями видимости, где располагаются имена, созданные вложенными операциями присваивания.


* Подобно именам в модуле, имена, созданные внутри инструкции `class`, становятся атрибутами объекта класса.

Основное **отличие классов в том, что их пространства имен также составляют основу механизма наследования** - ссылки на атрибуты, отсутствующие в классе или объекте экземпляра, будут получены из других классов.

>Все инструкции внутри инструкции `class` выполняются, когда выполняется сама инструкция `class` (а не когда позднее класс вызывается для создания экземпляра)

Присваивание объекта, не являющегося функцией, создает **атрибуты данных**, совместно используемые всеми экземплярами:

In [1]:
class SharedData:
    spam = 42
    
x = SharedData()
y = SharedData()
x.spam, y.spam

(42, 42)

Мы можем изменять значение атрибута, выполняя присваивание через имя класса, и обращаться к нему через имена экземпляров или класса.

In [2]:
SharedData.spam = 99
x.spam, y.spam, SharedData.spam

(99, 99, 99)

Операция присваивания, применяемая к  атрибуту экземпляра, создает или
изменяет имя в экземпляре, а не в классе.
>Поиск в дереве наследования производится только при попытке чтения атрибута, но не при присваивании: операция присваивания атрибуту объекта всегда изменяет сам объект, а не что-то другое (При условии, что класс не переопределил операцию присваивания с помощью метода перегрузки оператора `__setattr__`)

In [3]:
x.spam = 88
x.spam, y.spam, SharedData.spam

(88, 99, 99)

Ниже приводится более понятный пример этого поведения, где одно и то же имя создается в двух местах. Предположим, что мы используем следующий класс:

In [4]:
class MixedNames:              # Определение класса
    data = 'spam'              # Присваивание атрибуту класса
    
    def __init__(self, value): # Присваивание имени метода
        self.data = value      # Присваивание атрибуту экземпляра
        
    def display(self):
        print(self.data, MixedNames.data) # Атрибут экземпляра, атрибут класса

In [5]:
x = MixedNames(1)        # Создаются два объекта экземпляров,
y = MixedNames(2)        # каждый из которых имеет свой атрибут data
x.display(); y.display() # self.data - это другие атрибуты,
                         # а MixedNames.data - тот же самый

1 spam
2 spam


Суть этого примера состоит в  том, что атрибут `data` находится в  двух разных местах: в объектах экземпляров (создаются присваиванием атрибуту `self.data` в методе `__init__`) и в классе, от которого они наследуют имена (создается присваиванием имени `data` в инструкции `class`).

Метод класса `display` выводит обе версии – сначала атрибут экземпляра `self`, а затем атрибут класса.

>Атрибуты классов совместно используются всеми экземплярами, а атрибуты экземпляров уникальны для каждого экземпляра – ни данные, ни поведение экземпляра недоступны для совместного использования.

## Методы

Методы – это обычные объекты функций, которые создаются инструкциями `def` в теле инструкции `class`.

Методы реализуют поведение, наследуемое объектами экземпляров. С точки зрения программирования методы работают точно так же, как и обычные функции, с одним важным исключением: **в первом аргументе методам всегда передается подразумеваемый объект экземпляра**.

Вызов метода экземпляра:
```
instance.method(args...)
```

автоматически преобразуется в вызов метода класса:
```
class.method(instance, args...)
```
где класс определяется в результате поиска имени метода по дереву наследования.

Первый аргумент в  методах классов обычно называется **`self`**, в  соответствии с  общепринятыми соглашениями (с технической точки зрения **само имя не играет никакой роли, значение имеет позиция аргумента**).

Этот аргумент обеспечивает доступ к экземпляру, то есть к субъекту вызова, – поскольку из классов может создаваться множество объектов экземпляров, этот аргумент необходим для доступа к данным конкретного экземпляра.

### Пример метода

In [6]:
class NextClass:              # Определение класса
    def printer(self, text):  # Определение метода
        self.message = text   # Изменение экземпляра
        print(self.message)   # Обращение к экземпляру

In [7]:
x = NextClass()             # Создать экземпляр
x.printer('instance call')  # вызвать его метод

instance call


In [8]:
x.message  # экземпляр изменился

'instance call'

Когда метод вызывается с использованием квалифицированного имени экземпляра, как в данном случае, то сначала определяется местонахождение метода `printer`, а затем его аргументу `self` автоматически присваивается объект экземпляра (`x`).

Методы могут вызываться любым из двух способов – через экземпляр или через сам класс

In [9]:
NextClass.printer(x, 'class call')  # прямой вызов метода класса

class call


In [10]:
x.message  # экземпляр снова изменился

'class call'

### Вызов конструкторов суперклассов

Обычно методы вызываются через экземпляры. Тем не менее вызовы методов через имя класса могут играть особую роль. Одна из таких ролей связана с вызовом конструктора.

Метод `__init__` наследуется точно так же, как и  любые другие атрибуты. Это означает, что во время создания экземпляра интерпретатор отыскивает только один метод `__init__`. Если в конструкторе подкласса необходимо гарантировать выполнение действий, предусматриваемых конструктором суперкласса, необходимо явно вызвать метод `__init__` через имя класса:
```
class Super:
    def __init__(self, x):
        ...программный код по умолчанию...
    
class Sub(Super):
    def __init__(self, x, y):
        Super.__init__(self, x)  # Вызов __init__ суперкласса
        ...адаптированный код... # Выполнить доп. действия

I = Sub(1, 2)
```

Вызывать конструктор суперкласса таким способом следует, только если это действительно необходимо, – без этого вызова подкласс полностью переопределяет данный метод.

>Внутри одного и того же класса можно определить несколько методов с именем `__init__`, но **использоваться будет только последнее определение**.

### Другие возможности методов

Такой способ вызова методов через имя класса представляет собой основу для расширения (без полной замены) поведения унаследованных методов. 

Существуют также **статические методы**, которые не предполагают наличие объекта экземпляра в  первом аргументе. Такие методы могут действовать как обычные функции, имена которых являются локальными по отношению к классам, где они были определены, и использоваться для манипулирования данными класса.

Родственные им **методы класса** принимают в первом аргументе сам класс, а не экземпляр, и могут использоваться для манипулирования данными, принадлежащими конкретному классу. Однако это дополнительное расширение не является обязательным – обычно нам всегда бывает необходимо передавать экземпляр методам, вызываемым либо через сам экземпляр, либо через имя класса.

## Наследование

В языке Python наследование вступает в игру после того, как объект будет квалифицирован, и его действие заключается в операции поиска в дереве определений атрибутов (в одном или более пространствах имен). 

Каждый раз, когда используется выражение вида `object.attr` (где `object`  – это объект экземпляра или класса), интерпретатор приступает к  поиску первого вхождения атрибута `attr` в дереве пространств имен снизу вверх, начиная с объекта `object`. Сюда относятся и  ссылки на атрибуты аргумента `self` внутри методов.

### Создание дерева атрибутов

* Атрибуты экземпляров создаются посредством присваивания атрибутам аргумента `self` в методах.


* Атрибуты классов создаются инструкциями (присваивания), расположенными внутри инструкции `class`.


* Ссылки на суперклассы создаются путем перечисления классов в круглых скобках в заголовке инструкции `class`.

<img src='./images/6_28.png'>

### Специализация унаследованных методов

Только что описанная модель поиска в дереве наследования представляет собой прекрасный способ специализации программ. Поскольку механизм наследования сначала пытается отыскать имена в подклассах и только потом в их суперклассах, подклассы могут изменять поведение по умолчанию, предусматриваемое атрибутами их суперклассов.

Например, подклассы могут полностью **замещать** унаследованные атрибуты, **предоставлять атрибуты**, которые ожидается отыскать в суперклассах, и **расширять** методы суперкласса за счет их вызова из методов подкласса. Мы уже видели прием с замещением в действии.

Ниже приводится пример, демонстрирующий, как выполняется *расширение*:

In [11]:
class Super:
    def method(self):
        print('in Super.method')
        
class Sub(Super):
    def method(self):                # Переопределить метод
        print('starting Sub.method') # Дополнительное действие
        Super.method(self)           # Выполнить действие по умолчанию
        print('ending Sub.method')

Главное здесь – это прямые вызовы методов суперкласса. Класс `Sub` замещает метод `method` класса `Super` своей собственной, специализированной версией. Но внутри замещающего метода в классе `Sub` производится вызов версии, экспортируемой классом `Super`, чтобы выполнить действия по умолчанию. Другими словами, метод `Sub.method` не замещает полностью метод `Super.method`, а просто расширяет его:

In [12]:
x = Super()  # Создать экземпляр класса Super
x.method()   # Вызвать Super.method

in Super.method


In [13]:
x = Sub()  # Создать экземпляр класса Sub
x.method() # Вызвать Sub.method, который вызовет Super.method

starting Sub.method
in Super.method
ending Sub.method


### Приемы организации взаимодействия классов

Различные приемы использования классов:

* **`Super`** определяет метод `method` и метод `delegate`, который предполагает наличие метода `action` в подклассе


* **`Interior`** не предоставляет никаких новых имен, поэтому он получает все, что определено в классе `Super`


* **`Replacer`** переопределяет метод `method` класса `Super` своей собственной версией


* **`Extender`** адаптирует метод `method` класса `Super`, переопределяя и вызывая его, чтобы выполнить действия, предусмотренные по умолчанию.


* **`Provider`** реализует метод `action`, который ожидается методом `delegate` класса `Super`.

In [14]:
class Super:
    def method(self):
        print('in Super.method')  # Поведение по умолчанию
    def delegate(self):
        self.action()             # Ожидаемый метод

class Inheritor(Super):           # Наследует методы, как они есть
    pass

class Replacer(Super):            # Полностью замещает method
    def method(self):
        print('in Replacer.method')

class Extender(Super):            # Расширяет поведение метода method
    def method(self):
        print('starting Extender.method')
        Super.method(self)
        print('ending Extender.method')

class Provider(Super):            # Определяет необходимый метод
    def action(self):
        print('in Provider.action')

for klass in (Inheritor, Replacer, Extender):
    print('\n' + klass.__name__ + '...')
    klass().method()
    
print('\nProvider...')
x = Provider()
x.delegate()


Inheritor...
in Super.method

Replacer...
in Replacer.method

Extender...
starting Extender.method
in Super.method
ending Extender.method

Provider...
in Provider.action


### Абстрактные суперклассы

В предыдущем примере когда через экземпляр `Provider` вызывается метод `delegate`, инициируются **две независимые процедуры поиска**:

1. При вызове `x.delegate` интерпретатор отыскивает метод `delegate` в классе `Super`, начиная поиск от экземпляра `Provider` и двигаясь вверх по дереву наследования. Экземпляр `x` передается методу в виде аргумента `self`, как обычно.

2. Внутри метода `Super.delegate` выражение `self.action` приводит к запуску нового, независимого поиска в дереве наследования, начиная от экземпляра `self` и дальше вверх по дереву. Поскольку аргумент `self` ссылается на экземпляр класса `Provider`, метод `action` будет найден в подклассе `Provider`.

Такие суперклассы (в терминах метода `delegate`), как в этом примере, иногда называют
>**абстрактными суперклассами** - классы, которые предполагают, что часть их функциональности будет реализована их подклассами

Если ожидаемый метод не определен в подклассе, интерпретатор возбудит исключение. Разработчики классов иногда делают такие требования к подклассам более очевидными с помощью инструкций `assert` или возбуждая встроенное исключение `NotImplementedError` с помощью инструкции `raise`:

In [15]:
class Super:
    def delegate(self):
        self.action()
    def action(self):
        # При вызове этой версии
        assert False, 'action must be defined!'  
        
x = Super()
x.delegate()

AssertionError: action must be defined!

In [16]:
class Super:
    def delegate(self):
        self.action()
    def action(self):
        # возбуждение исключения напрямую
        raise NotImplementedError('action must be defined!')  
        
x = Super()
x.delegate()

NotImplementedError: action must be defined!

### Абстрактные суперклассы в Python 2.6 и 3.0

В версиях Python  2.6 и  3.0 **абстрактные суперклассы (они же «абстрактные базовые классы»)**, представленные в  предыдущем разделе, которые требуют, чтобы подклассы переопределяли некоторые методы, могут быть реализованы с применением специальной синтаксической конструкции определения класса.

В Python 3.0 для этих целей **используется именованный аргумент в заголовке инструкции `class` и специальный декоратор `@abstract` методов**:
```
from abc import ABCMeta, abstractmethod

class Super(metaclass=ABCMeta):
    @abstractmethod
    def method(self, ...):
        pass
```
Реализованный таким способом класс с  абстрактным методом не может использоваться для создания экземпляров (то есть нам не удастся создать экземпляр вызовом этого класса), если все абстрактные методы не будут реализованы в  подклассах.

Хотя при такой реализации объем программного кода увеличивается, тем не менее она имеет свои преимущества – ошибки из-за отсутствующих методов будут появляться при попытке создать экземпляр класса, а не позднее, при попытке вызвать отсутствующий метод.

Данная возможность может использоваться для построения ожидаемого интерфейса, полнота реализации которого будет автоматически проверяться в клиентских классах.

Ниже пример:

In [17]:
from abc import ABCMeta, abstractmethod

class Super(metaclass=ABCMeta):
    def delegate(self):
        self.action()
    
    @abstractmethod
    def action(self):
        pass

In [18]:
x = Super()

TypeError: Can't instantiate abstract class Super with abstract methods action

In [19]:
class Sub(Super): pass

x = Sub()

TypeError: Can't instantiate abstract class Sub with abstract methods action

In [20]:
class Sub(Super):
    def action(self):
        print('spam')
        
x = Sub()
x.delegate()

spam


## Пространства имен: окончание истории

Квалифицированные и неквалифицированные имена интерпретируются по-разному, и некоторые области видимости служат для инициализации пространств имен объектов:

* Неквалифицированные имена (например, `X`) располагаются в областях видимости.


* Квалифицированные имена атрибутов (например, `object.X`) принадлежат пространствам имен объектов.


* Некоторые области видимости инициализируют пространства имен объектов (в модулях и классах)

### Простые имена: глобальные, пока не выполняется присваивание

Поиск неквалифицированных простых имен выполняется в соответствии с правилом лексической видимости **LEGB**

* **Присваивание (`X = value`)** Операция присваивания делает имена локальными: создает или изменяет имя `X` в текущей локальной области видимости, если имя не объявлено глобальным.


* **Ссылка (`X`)** пытается отыскать имя `X` в текущей локальной области видимости, затем в области видимости каждой из вмещающих функций, затем в текущей глобальной области видимости и, наконец, во встроенной области видимости.

### Имена атрибутов: пространства имен объектов

Квалифицированные имена атрибутов ссылаются на атрибуты конкретных объектов и к ним применяются правила, предназначенные для модулей и классов. Для объектов классов и экземпляров эти правила дополняются включением процедуры поиска в дереве наследования:

* **Присваивание (`object.X = value`)** создает или изменяет атрибут `X` в пространстве имен объекта `object`, и ничего больше. Восхождение по дереву наследования происходит только при попытке получить ссылку на атрибут, но не при выполнении операции присваивания.


* **Ссылка (`object.X`)** для объектов, созданных на основе классов, поиск атрибута `X` производится сначала в объекте `object`, затем во всех классах, расположенных выше в дереве наследования. В случае объектов, которые создаются не из классов, таких как модули, атрибут `X` извлекается непосредственно из объекта `object`.

### «Дзен» пространств имен в Python: классификация имен происходит при присваивании

Из-за различий в процедурах поиска простых и составных имен и нескольких уровней поиска в  обеих процедурах иногда бывает трудно сказать, где будет найдено имя.

В  языке Python **место, где выполняется присваивание**, имеет крайне важное значение – оно **полностью определяет область видимости или
объект, где будет размещаться имя**.

In [21]:
# manynames.py
X = 11 # Глобальное (в модуле) имя/атрибут (X, или manynames.X)

def f():
    print(X) # Обращение к глобальному имени X (11)
    
def g():
    X = 22 # Локальная (в ф-ии) переменная (X, скрывает имя X в модуле)
    print(X)

class C:
    X = 33           # Атрибут класса (C.X)
    def m(self):
        X = 44       # Локальная переменная в методе (X)
        self.X = 55  # Атрибут экземпляра (instance.X)

In [22]:
print(X) # 11: модуль (за пределами файла manynames.X)

11


In [23]:
f() # 11: глобальная

11


In [24]:
g() # 22: локальная

22


In [25]:
print(X) # 11: переменная модуля не изменилась

11


In [26]:
obj = C()    # Создать экземпляр
print(obj.X) # 33: переменная класса, унаследованная экземпляром

33


In [27]:
obj.m()       # Присоединить атрибут X к экземпляру
print(obj.X)  # 55: экземпляр

55


In [28]:
print(C.X)    # 33: класс (она же obj.X, если в экземпляре нет X)

33


In [29]:
print(C.m.X)  # ОШИБКА: видима только в методе

AttributeError: 'function' object has no attribute 'X'

In [30]:
print(g.X)    # ОШИБКА: видима только в функции

AttributeError: 'function' object has no attribute 'X'

In [31]:
class C:
    X = 33           # Атрибут класса (C.X)
    def m(self):
        X = 44       # Локальная переменная в методе (X)
        self.X = 55  # Атрибут экземпляра (instance.X)
    m.X = 99         # Атрибут функции, будет виден через вызов C.m.X
    
C.m.X

99

### Словари пространств имен

**Пространства имен модулей, объектов классов и экземпляров** фактически **реализованы как словари** и доступны в виде встроенного **атрибута `__dict__`**

Чтобы понять внутреннее устройство атрибутов, давайте с  помощью интерактивной оболочки проследим, как растут словари пространств имен, когда в игру вступают классы

In [32]:
class Super:
    def hello(self):
        self.data1 = 'spam'

class Sub(Super):
    def hola(self):
        self.data2 = 'eggs'

Когда мы создаем экземпляр подкласса, он начинает свое существование с пустым словарем пространства имен, но имеет ссылку на класс, стоящий выше в дереве наследования. Фактически дерево наследования доступно в виде специальных атрибутов, которые вы можете проверить.

**Экземпляры обладают атрибутом `__class__`**, который ссылается на класс, а  **классы имеют атрибут `__bases__`**, который является кортежем, содержащим ссылки на суперклассы выше в  дереве наследования

In [33]:
X = Sub()
X.__dict__  # Словарь пространства имен экземпляра

{}

In [34]:
X.__class__  # Класс экземпляра

__main__.Sub

In [35]:
Sub.__bases__  # Суперклассы данного класса

(__main__.Super,)

In [36]:
Super.__bases__

(object,)

Так как в  классах выполняется присваивание атрибутам аргумента `self`, тем самым они заполняют объекты экземпляров, то есть атрибуты включаются в словари пространств имен экземпляров, а  не классов. В  пространство имен объекта экземпляра записываются данные, которые могут отличаться для разных экземпляров, и  аргумент `self` является точкой входа в  это пространство имен:

In [37]:
Y = Sub()
X.hello()
X.__dict__

{'data1': 'spam'}

In [38]:
X.hola()
X.__dict__

{'data1': 'spam', 'data2': 'eggs'}

In [39]:
Sub.__dict__.keys()

dict_keys(['__module__', 'hola', '__doc__'])

In [40]:
Super.__dict__.keys()

dict_keys(['__module__', 'hello', '__dict__', '__weakref__', '__doc__'])

In [41]:
Y.__dict__

{}

Второй экземпляр `Y` по-прежнему имеет пустой словарь пространства имен, несмотря на то, что словарь экземпляра `X` заполнялся инструкциями присваивания в методах - у каждого экземпляра имеется свой, независимый словарь, который изначально пуст.

Так как атрибуты фактически являются ключами словаря, существует **два
способа получать и  изменять их значения  – по квалифицированным именам
или индексированием по ключу** Однако эквивалентность доступа по ключу применяется только к атрибутам, фактически присоединенным к  экземпляру:

In [42]:
X.data1, X.__dict__['data1']

('spam', 'spam')

In [43]:
X.data3 = 'toast'
X.__dict__

{'data1': 'spam', 'data2': 'eggs', 'data3': 'toast'}

In [44]:
X.__dict__['data3'] = 'ham'
X.data3

'ham'

Унаследованный атрибут `X.hello` недоступен через доступ по ключу:

In [45]:
X.__dict__['hello']

KeyError: 'hello'

In [46]:
X.hello

<bound method Super.hello of <__main__.Sub object at 0x7ff24c3917b8>>

**Функция `dir`** применяется к объектам, имеющим атрибуты, она сортирует свой список и включает в него некоторые системные атрибуты, а также автоматически собирает унаследованные атрибуты и добавляет в перечень имена, унаследованные от класса `object`, который является суперклассом для всех классов.

In [47]:
X.__dict__

{'data1': 'spam', 'data2': 'eggs', 'data3': 'ham'}

In [48]:
dir(X)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data1',
 'data2',
 'data3',
 'hello',
 'hola']

### Ссылки на пространства имен

Специальные **атрибуты экземпляра и класса `__class__` и `__bases__` позволяют осматривать иерархии наследования** в вашем программном коде. Например, их можно использовать для отображения дерева классов:

In [49]:
# classtree.py
"""
Выполняет обход дерева наследования снизу вверх, используя ссылки на пространства
имен, и отображает суперклассы с отступами
"""
def classtree(cls, indent):
    print('.' * indent + cls.__name__) # Вывести имя класса
    for supercls in cls.__bases__:  # Рекурсивный обход всех суперклассов
        # Каждый суперкласс может быть посещен более одного раза
        classtree(supercls, indent+3) 

def instancetree(inst):
    print('Tree of', inst)  # Показать экземпляр
    classtree(inst.__class__, 3) # Взойти к его классу

def selftest():
    class A: pass
    class B(A): pass
    class C(A): pass
    class D(B,C): pass
    class E: pass
    class F(D,E): pass
    instancetree(B())
    instancetree(F())

if __name__ == '__main__': selftest()

Tree of <__main__.selftest.<locals>.B object at 0x7ff24c338f28>
...B
......A
.........object
Tree of <__main__.selftest.<locals>.F object at 0x7ff24c338f28>
...F
......D
.........B
............A
...............object
.........C
............A
...............object
......E
.........object


Функция `classtree` в этом сценарии является рекурсивной – она выводит имя класса, используя атрибут `__name__`, и затем начинает подъем к суперклассам, вызывая саму себя. Это позволяет функции выполнять обход деревьев классов произвольной формы – в процессе рекурсии выполняется подъем по дереву и  заканчивается по достижении корневых суперклассов, у  которых атрибут `__bases__` пуст.

In [50]:
object.__bases__

()

## Еще раз о строках документирования

Пример модуля в  предыдущем разделе содержит строку документирования, описывающую этот модуль, но точно так же они могут использоваться для описания компонентов классов. Строки документирования, которые мы подробно рассматривали в главе 15, – это литералы строк, которые присутствуют на верхнем уровне различных структур и автоматически сохраняются интерпретатором в  атрибутах `__doc__` соответствующих им объектов.

Строки документирования могут присутствовать в  модулях, в  инструкциях `def`, а  также в определениях классов и методов.

Основное преимущество строк документирования состоит в том, что их содержимое доступно во время выполнения. То есть, если текст был оформлен в виде строки документирования, можно будет обратиться к атрибуту `__doc__` объекта, чтобы получить его описание

In [51]:
import sys
sys.path.append('./exercises/6_28/')
import classtree

In [52]:
classtree.__doc__

'\nВыполняет обход дерева наследования снизу вверх, используя ссылки на пространства\nимен, и отображает суперклассы с отступами\n'

## Классы и модули

Так как классы и модули представляют собой пространства имен, различия между ними бывает трудно заметить сразу.

В двух словах:

* Модули
    * Это пакеты данных и исполняемого кода.
    
    * Создаются как файлы с программным кодом на языке Python или как расширения на языке C.
    
    * Задействуются операцией импортирования.
    
    
* Классы
    * Реализуют новые объекты.
    
    * Создаются с помощью инструкции `class`
    
    * Задействуются операцией вызова.
    
    * Всегда располагаются внутри модуля.
    
Кроме того, классы поддерживают дополнительные возможности, недоступные в модулях, такие как перегрузка операторов, создание множества экземпляров и наследование.